In [2]:
### Standard Imports - Sorry PEP8 fans, do not look below
import pandas as pd, numpy as np, os, re, json, sys
from pathlib import Path
from datetime import datetime
from pathlib import Path
import os
import numpy as np
import pandas as pd

import random
from joshberry.utils import *
from joshberry.feats import *

## Specific Imports



In [3]:
### set path directories
curr_dir = Path(os.getcwd())
data_dir = Path(curr_dir.parents[0] / 'data/')
artifacts_dir = Path(curr_dir.parents[0] / 'artifacts/')

### Helper Functions

In [11]:
def add_new_rows(data):
    orig_rows = len(data)
    data['simulate_n'] = np.random.poisson(1, orig_rows) + 1
    distinct_n = data.simulate_n.unique()
    
    result=[]
    for n in distinct_n:
        temp = df[df['simulate_n']==n]
        
        for j in range(1,n+1):
            if len(result) == 0:
                result = temp
            else:
                result = pd.concat([result, temp])
            
    return result.reset_index(drop=True)

def random_date(x):
    s_d = pd.to_datetime(x[x['start_date'].notna()]['start_date'])
    e_d = pd.to_datetime(x[x['end_date'].notna()]['end_date'])
    
    start_u = s_d.iloc[0].value//10**9
    end_u = e_d.iloc[0].value//10**9
    try:
        end_date_list = pd.to_datetime(np.random.randint(start_u, end_u, len(x)-1), unit='s').values
        end_date_list.sort()
    except:
        print(x)
    end_date_list = np.append(end_date_list, e_d.values)
    x['end_date'] = end_date_list
    mask = x['start_date'].isna()
    x.loc[mask,'start_date'] = x.shift(1).loc[mask]['end_date'].astype(str)
    x['start_date'] = pd.to_datetime(x['start_date']).dt.date
    x['end_date'] = pd.to_datetime(x['end_date']).dt.date
    return x

### Generating Data

In [12]:
seed = 20171230
np.random.seed(seed)
output_filename = 'customer_account.csv'

In [13]:
n_rows = 5000

#! Generate new dataframe
df = pd.DataFrame(np.random.randint(0, 2147483647, size=(n_rows, 1)).astype('str'), columns=['AcctID'])
df['AcctID'] = 'C' + df['AcctID']

# export Customer ID for use later
df.to_csv(Path(data_dir) / 'temp_CustIDs.csv', index=False)

# expand to mutiple rows for some customers to simulate account activity
df = pd.DataFrame(add_new_rows(df)['AcctID'])
n_rows = len(df)

df = df.sort_values(['AcctID']).reset_index(drop=True)

# assign an ordering as a helper column
df['__rank'] = df.groupby(['AcctID']).cumcount()+1
df['__maxrank'] = df.groupby('AcctID')['__rank'].transform('max')

# always alternate ACTIVE and INACTIVE
df['account_status'] = np.where(df['__rank'] % 2, 'ACTIVE', 'INACTIVE')

# arbitrary start date + random buffer for first start
df['__dt'] = pd.to_datetime('2015-01-01')
df['__buffer'] = np.round(np.random.gamma(2, 60, n_rows), decimals=0).astype('int64')

# end date is easy for the final status
df['end_date'] = np.where(df['__rank']==df['__maxrank']
                         ,'2021-04-01'
                         ,pd.NaT)

# start date is easy for the first status (arbitrary + random buffer)
df['start_date'] = pd.to_datetime(np.where(df['__rank'] == 1
                                            ,df['__dt'] + (pd.to_timedelta(df['__buffer'], unit='D')*1.5)
                                            ,pd.NaT
                                           )
                                 )

# convert to date formats
for dtcol in ['end_date','start_date','__dt']:
    df[dtcol] = df[dtcol].dt.strftime('%Y-%m-%d')



# only need to simulate ones that have more than 1 entry
needs_sim = df['__maxrank']>1

dfA = df[needs_sim]
dfB = df[~needs_sim]

# simulate using a custom function
dfA = dfA.groupby('AcctID').apply(random_date)

# merge them back together
df = pd.concat([dfA, dfB], axis=0)

# clean up helper columns
df = df[df.columns.drop(list(df.filter(regex='__')))]

df.to_csv(Path(data_dir) / output_filename, index=False, chunksize=chunky(df))

Chunky optimized your write speed by using a chunksize of 6707.


In [14]:
dfA

,AcctID,__rank,__maxrank,account_status,__dt,__buffer,end_date,start_date
0,C1000089533,1,2,ACTIVE,2015-01-01,103,2020-04-09,2015-06-04
1,C1000089533,2,2,INACTIVE,2015-01-01,15,2021-04-01,2020-04-09
2,C1000410568,1,3,ACTIVE,2015-01-01,282,2016-04-09,2016-02-28
3,C1000410568,2,3,INACTIVE,2015-01-01,50,2016-10-09,2016-04-09
4,C1000410568,3,3,ACTIVE,2015-01-01,79,2021-04-01,2016-10-09
...,...,...,...,...,...,...,...,...
10093,C998017489,2,2,INACTIVE,2015-01-01,57,2021-04-01,2018-07-21
10094,C998107319,1,2,ACTIVE,2015-01-01,139,2019-08-02,2015-07-28
10095,C998107319,2,2,INACTIVE,2015-01-01,292,2021-04-01,2019-08-02
10097,C998991775,1,2,ACTIVE,2015-01-01,76,2015-11-16,2015-04-25
